In [ ]:
# 참고 : https://github.com/kairess/mask-detection

In [1]:
# 내 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 필요한 패키지와 모듈을 불러옴
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import time
import io
import base64
from IPython.display import HTML

In [3]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('gdrive/My Drive/CV2/Face Mask Detection/imgs/04.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [4]:
# DNN으로 학습한 모델(caffemodel)을 사용
model_name = "gdrive/My Drive/CV2/Face Mask Detection/models/res10_300x300_ssd_iter_140000.caffemodel"
# model Architecture 의 설계도 파일
prototxt_name = "gdrive/My Drive/CV2/Face Mask Detection/models/deploy.prototxt"

# Mask Detection 모델을 사용
masknet = load_model('gdrive/My Drive/CV2/Face Mask Detection/models/mask_detector.model')

min_confidence = 0.4  # detection 으로 인정할 최소 확률(신뢰도)
file_name = "gdrive/My Drive/CV2/Face Mask Detection/imgs/04.mp4"   # 원본 동영상
output_name = 'output_video.mp4'    # detection 된 output 동영상

In [5]:
def detectAndDisplay(frame):
    height, width = frame.shape[:2]

    # caffemodel의 weight 값과 모델 네트워크 구성을 불러와서 모델을 정의한다.
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)

    # 이미지를 300x300 으로 size를 조정하고 blob 를 만든다.
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))

    # blob을 모델에 넣는다
    model.setInput(blob)
    # detection을 수행한다.
    detections = model.forward()

    # detections 한 수만큼 루프가 돈다.
    for i in range(0, detections.shape[2]):
           
            confidence = detections[0, 0, i, 2]  # confidence 는 detection한 확률을 나타냄

            # min_confidence 보다 큰 경우에만 detection 으로 인정함
            if confidence > min_confidence:
                    # detection 된 영역을 boxing
                    # 상대적 좌표 * np.array([width, height, width, height]) 절대적인 boxing 좌표을 구해낸다 
                    box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                    (startX, startY, endX, endY) = box.astype("int")
                    print(confidence, startX, startY, endX, endY)

                    faceROI = frame[startY:endY, startX:endX]

                    face_input = cv2.resize(faceROI, dsize=(224, 224))
                    face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
                    face_input = preprocess_input(face_input)
                    face_input = np.expand_dims(face_input, axis=0)  # 앞에 차원을 하나 늘려줌

                    # Face mask detection        
                    mask, nomask = masknet.predict(face_input).squeeze()   # squeeze() 는 차원 중 사이즈가 1인 것을 찾아 제거한다.

                    if mask > nomask:
                        color = (0, 255, 0)
                        label = 'Mask %d%%' % (mask * 100)
                    else:
                        color = (0, 0, 255)
                        label = 'No Mask %d%%' % (nomask * 100)

                    y = startY - 10 if startY - 10 > 10 else startY + 10
                    cv2.rectangle(frame, (startX, startY), (endX, endY),
                            (0, 255, 0), 2)
                    cv2.putText(frame, label, (startX, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        #fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (frame.shape[1], frame.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(frame)

In [6]:
# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)

0.99469477 1481 467 1683 731
0.8808542 883 585 1065 867
0.70420563 1174 434 1326 636
0.5395626 464 305 650 563
0.9947172 1481 467 1683 731
0.88429344 883 585 1066 868
0.72617775 1174 435 1325 636
0.5438643 464 305 650 564
0.9943718 1480 467 1683 731
0.8851044 883 584 1066 868
0.70502895 1173 434 1326 636
0.5713428 463 306 650 566
0.9939926 1480 467 1682 732
0.8880122 883 585 1066 867
0.7103853 1173 434 1327 637
0.5840887 462 307 650 567
0.9933634 1480 467 1681 731
0.8965783 884 585 1067 868
0.7152755 1173 435 1328 637
0.6139445 462 308 650 569
0.99247015 1480 467 1681 731
0.89809436 884 585 1068 868
0.7079455 1173 435 1328 638
0.6212449 462 309 650 570
0.992295 1480 467 1680 731
0.9027939 884 585 1068 868
0.7125696 1173 435 1328 638
0.5986437 463 310 650 572
0.9924172 1480 468 1680 732
0.9034425 885 586 1068 868
0.7083164 1173 435 1328 639
0.5877836 463 311 650 572
0.9919007 1480 468 1680 731
0.90484226 884 585 1068 868
0.69942594 1173 436 1328 639
0.57852894 463 311 650 571
0.9919601 